In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import keras
from math import sin, cos, pi
import cv2
import tensorflow as tf

In [ ]:
!nvidia-smi

In [ ]:
class config:
    horizontal_flip = False
    rotation_augmentation = True
    brightness_augmentation = True
    shift_augmentation = True
    random_noise_augmentation = True

    rotation_angles = [12]
    pixel_shifts = [12]
    NUM_EPOCHS = 150
    BATCH_SIZE = 64

In [ ]:
!ls

In [ ]:
!unzip -u "../input/facial-keypoints-detection/test.zip"

In [ ]:
!unzip -u "../input/facial-keypoints-detection/training.zip"

In [ ]:
%%time

train_file = "training.csv"
test_file = "test.csv"
idlookup_file = "../input/facial-keypoints-detection/IdLookupTable.csv"
train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)
idlookup_data = pd.read_csv(idlookup_file)

In [ ]:
def plot_sample(image, keypoint, axis, title):
    image = image.reshape(96, 96)
    axis.imshow(image, cmap='gray')
    axis.scatter(keypoint[0::2], keypoint[1::2], marker='x', s=20)
    plt.title(title) 

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
print("Length of train dataset: {}".format(len(train_data)))
print("Length of test dataset: {}".format(len(test_data)))

In [ ]:
train_data.isnull().sum()

In [ ]:
clean_train_data = train_data.dropna()
print(clean_train_data.shape)

unclean_train_data = train_data.fillna(method = 'ffill')
print(unclean_train_data.shape)

In [ ]:
def load_images(image_data) -> np.array:
    images = []
    for idx, sample in image_data.iterrows():
        image = np.array(sample['Image'].split(' '), dtype=int)
        image = np.reshape(image, (96,96,1))
        images.append(image)
    images = np.array(images)/255.
    return images

def load_keypoints(keypoint_data) -> np.array:
    keypoint_data = keypoint_data.drop('Image',axis = 1)
    keypoint_features = []
    for idx, sample_keypoints in keypoint_data.iterrows():
        keypoint_features.append(sample_keypoints)
    keypoint_features = np.array(keypoint_features, dtype = 'float')
    return keypoint_features

In [ ]:
%%time

clean_train_images = load_images(clean_train_data)
print("Shape of clean_train_images: {}".format(clean_train_images.shape))
clean_train_keypoints = load_keypoints(clean_train_data)
print("Shape of clean_train_keypoints: {}".format(clean_train_keypoints.shape))
test_images = load_images(test_data)
print("Shape of test_images: {}".format(test_images.shape))

train_images = clean_train_images
train_keypoints = clean_train_keypoints

fig, axis = plt.subplots()
plot_sample(train_images[0], train_keypoints[0], axis, 'Sample Image and Keypoints')

unclean_train_images = load_images(unclean_train_data)
print("Shape of unclean_train_images {}".format(unclean_train_images.shape))
unclean_train_keypoints = load_keypoints(unclean_train_data)
print("Shape of unclean_train_keypoints {}".format(unclean_train_keypoints.shape))
train_images = np.concatenate((train_images, unclean_train_images))
train_keypoints = np.concatenate((train_keypoints, unclean_train_keypoints))

In [ ]:
def left_right_flip(images, keypoints):
    flipped_keypoints = []
    flipped_images = np.flip(images, axis=2)
    for idx, sample_keypoints in enumerate(keypoints):
        flipped_keypoints.append([96.-coor if idx%2==0 else coor for idx, coor in enumerate(sample_keypoints)])
    return flipped_images, flipped_keypoints

if config.horizontal_flip:
    flipped_train_images, flipped_train_keypoints = left_right_flip(clean_train_images, clean_train_keypoints)
    print("Shape of flipped_train_images: {}".format(flipped_train_images.shape))
    print("Shape of flipped_train_keypoints: {}".format(np.shape(flipped_train_keypoints)))
    train_images = np.concatenate((train_images, flipped_train_images))
    train_keypoints = np.concatenate((train_keypoints, flipped_train_keypoints))
    fig, axs = plt.subplots()
    plot_sample(flipped_train_images[0], flipped_train_keypoints[0], axs, "Horizontally flipped")

In [ ]:
def rotate_augmentation(images, keypoints):
    rotated_images, rotated_keypoints = [], []
    print("Augmentation for angles (in degrees): ")
    for angle in config.rotation_angles:
        for angle in [-angle, angle]:
            print(f'{angle}', end=' ')
            M = cv2.getRotationMatrix2D((48, 48), angle, 1.0)
            angle_rad = -angle*pi/180
            for image in images:
                rotated_image = cv2.warpAffine(image, M, (96, 96), flags=cv2.INTER_CUBIC)
                rotated_images.append(rotated_image)
            
            for keypoint in keypoints:
                rotated_keypoint = keypoint - 48 # shift points in the plane so the center of rotation is the origin
                
                for idx in range(0, len(rotated_keypoint), 2):
                    rotated_keypoint[idx] = rotated_keypoint[idx]*cos(angle_rad) - rotated_keypoint[idx+1]*sin(angle_rad)
                    rotated_keypoint[idx+1] = rotated_keypoint[idx]*sin(angle_rad) + rotated_keypoint[idx+1]*cos(angle_rad)
                rotated_keypoint += 48
                rotated_keypoints.append(rotated_keypoint)
    print(np.shape(rotated_images))
    return np.reshape(rotated_images, (-1, 96, 96, 1)), rotated_keypoints

if config.rotation_augmentation:
    rotated_train_images, rotated_train_keypoints = rotate_augmentation(clean_train_images, clean_train_keypoints)
    print("Shape of rotated_images {}".format(np.shape(rotated_train_images)))
    print("Shape of rotated_keypoints {}".format(np.shape(rotated_train_keypoints)))
    train_images = np.concatenate((train_images, rotated_train_images))
    train_keypoints = np.concatenate((train_keypoints, rotated_train_keypoints))
    fig, axs = plt.subplots()
    plot_sample(rotated_train_images[0], rotated_train_keypoints[0], axs, "Rotated Image")

In [ ]:
def alter_brightness(images, keypoints):
    altered_brightness_images = []
    inc_brightness_images = np.clip(images*1.2, 0, 1)
    dec_brightness_images = np.clip(images*0.8, 0, 1)
    altered_brightness_images.extend(inc_brightness_images)
    altered_brightness_images.extend(dec_brightness_images)
    return altered_brightness_images, np.concatenate((keypoints, keypoints))

if config.brightness_augmentation:
    alter_brightness_train_images, alter_brightness_train_keypoints = alter_brightness(clean_train_images, clean_train_keypoints)
    print("Shape of alter_brightness_train_images {}".format(np.shape(alter_brightness_train_images)))
    print("Shape of alter_brightness_train_keypoints {}".format(np.shape(alter_brightness_train_keypoints)))
    train_images = np.concatenate((train_images, alter_brightness_train_images))
    train_keypoints = np.concatenate((train_keypoints, alter_brightness_train_keypoints))
    fig, axs = plt.subplots()
    plot_sample(alter_brightness_train_images[0], alter_brightness_train_keypoints[0], axs, "Altered Brightness")

In [ ]:
def shift_images(images, keypoints):
    shifted_images, shifted_keypoints = [], []
    for shift in config.pixel_shifts:
        for (shift_x, shift_y) in [(-shift,-shift), (-shift,shift), (shift,-shift), (shift,shift)]:
            M = np.array([[1, 0, shift_x], [0, 1, shift_y]], dtype=float)
            for image, keypoint in zip(images, keypoints):
                shifted_image = cv2.warpAffine(image, M, (96, 96), flags=cv2.INTER_CUBIC)
                shifted_keypoint = np.array([(point+shift_x) if idx%2==0 else (point+shift_y) for idx, point in enumerate(keypoint)])
                if np.all(0.0<shifted_keypoint) and np.all(shifted_keypoint<96.0):
                    shifted_images.append(shifted_image.reshape(96, 96, 1))
                    shifted_keypoints.append(shifted_keypoint)
        shifted_keypoints = np.clip(shifted_keypoints, 0.0, 96.0)
        return shifted_images, shifted_keypoints
    
if config.shift_augmentation:
    shifted_train_images, shifted_train_keypoints = shift_images(clean_train_images, clean_train_keypoints)
    print("Shape of shifted_train_images: {}".format(np.shape(shifted_train_images)))
    print("Shape of shifted_train_keypoints: {}".format(np.shape(shifted_train_keypoints)))
    train_images = np.concatenate((train_images, shifted_train_images))
    train_keypoints = np.concatenate((train_keypoints, shifted_train_keypoints))
    fig, axs = plt.subplots()
    plot_sample(shifted_train_images[0], shifted_train_keypoints[0], axs, "Shifted Image")

In [ ]:
def add_noise(images):
    noisy_images = []
    for image in images:
        noisy_image = cv2.add(image, 0.008 * np.random.randn(96, 96, 1))
        noisy_images.append(np.reshape(noisy_image, (96, 96, 1)))
    return noisy_images

if config.random_noise_augmentation:
    noisy_train_images = add_noise(clean_train_images)
    print("Shape of noisy_train_images {}".format(np.shape(noisy_train_images)))
    train_images = np.concatenate((train_images, noisy_train_images))
    train_keypoints = np.concatenate((train_keypoints, clean_train_keypoints))
    fig, axs = plt.subplots()
    plot_sample(noisy_train_images[0], clean_train_keypoints[0], axs, "Noisy Image")

In [ ]:
print("Shape of train_image: {}".format(np.shape(train_images)))
print("Shape of train_keypoints: {}".format(np.shape(train_keypoints)))

In [ ]:
pretrained_model = tf.keras.applications.ResNet50(input_shape=(96, 96, 3), 
                                                        include_top=False, 
                                                        weights='imagenet')
pretrained_model.trainable = True

layers = [
    tf.keras.layers.Conv2D(filters=3,
                           kernel_size=1,
                           padding='same',
                           input_shape=(96, 96, 1)),
    tf.keras.layers.LeakyReLU(alpha=0.1),
    pretrained_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(30),
]

model = tf.keras.Sequential(layers, name='keypoints_regression')

In [ ]:
if os.path.isdir('trained_model'):
    model = tf.keras.models.load_model('trained_model')
    train_model = False
else:
    train_model = True

train_model = True

print("Training model: {}".format(train_model))

In [ ]:
if train_model:
    es = keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=30,
        verbose=1,
        mode='min',
        baseline=None,
        restore_best_weights=True)

    rlp = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-15,
        mode='min',
        verbose=1)
    
    mc = tf.keras.callbacks.ModelCheckpoint(
        filepath='model.{epoch:02d}-{val_loss:.4f}.h5', 
        save_freq='epoch', verbose=1, monitor='val_loss', 
        save_weights_only=True, save_best_only=True
    )    

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'acc'])

    history = model.fit(x=train_images,
                        y=train_keypoints,
                        epochs=config.NUM_EPOCHS,
                        batch_size=config.BATCH_SIZE,
                        validation_split=0.05,
                        callbacks=[es, rlp, mc])

In [ ]:
model.summary()

In [ ]:
if train_model:
    sns.set_style('darkgrid')

    fig, ax = plt.subplots(3, 1, figsize=(20, 10))
    df = pd.DataFrame(history.history)
    df[['mae', 'val_mae']].plot(ax=ax[0])
    df[['loss', 'val_loss']].plot(ax=ax[1])
    df[['acc', 'val_acc']].plot(ax=ax[2])
    ax[0].set_title('Model MAE', fontsize=12)
    ax[1].set_title('Model Loss', fontsize=12)
    ax[2].set_title('Model Acc', fontsize=12)
    fig.suptitle('Modle Metrics', fontsize=18)

In [ ]:
%%time

test_preds = model.predict(test_images)

In [ ]:
a = test_preds[:, 22:24]
b = test_preds[:, 24:26]
mouth_length = np.linalg.norm(a-b, axis=1)
a = test_preds[:, 26:28]
b - test_preds[:, 28:30]
mouth_width = np.linalg.norm(a-b, axis=1)
proportions = np.abs(mouth_width/mouth_length)

In [ ]:
fig = plt.figure(figsize=(20, 16))
idxs = np.random.choice(test_preds.shape[0], 20)
print(idxs)
for i, idx in enumerate(idxs):
    axis = fig.add_subplot(4, 5, i+1, xticks=[], yticks=[])
    plot_sample(test_images[idx], test_preds[idx], axis, str(proportions[idx]))

In [ ]:
if train_model:
    feature_names = list(idlookup_data['FeatureName'])
    image_ids = list(idlookup_data['ImageId']-1)
    row_ids = list(idlookup_data['RowId'])

    feature_list = []
    for feature in feature_names:
        feature_list.append(feature_names.index(feature))

    predictions = []
    for x,y in zip(image_ids, feature_list):
        predictions.append(test_preds[x][y])

    row_ids = pd.Series(row_ids, name = 'RowId')
    locations = pd.Series(predictions, name = 'Location')
    locations = locations.clip(0.0,96.0)
    submission_result = pd.concat([row_ids,locations],axis = 1)
    submission_result.to_csv('submission.csv',index = False)

In [ ]:
if train_model:
    model.save('trained_model')

In [ ]:
!ls